<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/yeappppppy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Load the data
df = pd.read_csv('stock_mothly_returns.csv')
df=df.dropna()
# Preprocess the data
df['date'] = pd.to_datetime(df['date'])
df['outperform'] = (df['weekly_return'] > 0).astype(int)
# Encode ticker as numeric
label_encoder = LabelEncoder()
df['ticker_encoded'] = label_encoder.fit_transform(df['ticker'])

# Select features for the model
features = ['ticker_encoded','dollar_volume', 'adj close', 'atr', 'bb_high', 'bb_low', 'bb_mid',
            'garman_klass_vol', 'macd', 'rsi', 'return_1m', 'return_2m',
            'return_3m', 'return_6m', 'return_9m', 'return_12m']

X = df[features]
y = df['outperform']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test_scaled)

# Get probability estimates
y_prob = rf_classifier.predict_proba(X_test_scaled)

# Create a DataFrame with test data, predictions, and probabilities
results_df = pd.DataFrame({
    'actual': y_test,
    'predicted': y_pred,
    'prob_underperform': y_prob[:, 0],
    'prob_outperform': y_prob[:, 1],

})



# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_classifier.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)
# Add original features to the results DataFrame
results_df = pd.concat([results_df, X_test.reset_index(drop=True)], axis=1)

# Function to predict for new data
def predict_performance(new_data):
    new_data_scaled = scaler.transform(new_data)
    prediction = rf_classifier.predict(new_data_scaled)
    return "Outperform" if prediction[0] == 1 else "Underperform"
# Function to predict and rank stocks


<ipython-input-26-dd7c0f954e1a>:13: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(df['date'])


Confusion Matrix:
[[ 7 21]
 [ 5 33]]

Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.25      0.35        28
           1       0.61      0.87      0.72        38

    accuracy                           0.61        66
   macro avg       0.60      0.56      0.53        66
weighted avg       0.60      0.61      0.56        66


Feature Importance:
             feature  importance
3                atr    0.081358
13         return_6m    0.079563
1      dollar_volume    0.078465
9                rsi    0.075976
10         return_1m    0.071895
15        return_12m    0.068849
12         return_3m    0.066847
7   garman_klass_vol    0.065993
8               macd    0.064106
14         return_9m    0.062156
11         return_2m    0.058893
5             bb_low    0.057325
4            bb_high    0.056080
6             bb_mid    0.055400
2          adj close    0.046770
0     ticker_encoded    0.010323


In [23]:
# Example usage with test data
outperform_ranked = predict_performance(X_test)
print("Top 10 stocks predicted to outperform, ranked by confidence:")
print(outperform_ranked)


Top 10 stocks predicted to outperform, ranked by confidence:
Outperform


In [29]:
results_df.dropna(inplace=True)

In [33]:
results_df['final_names']=label_encoder.inverse_transform(results_df['ticker_encoded'].astype(int))

In [25]:
results_df.to_csv('results.csv', index=False)

In [42]:
results_df.sort_values(by='prob_outperform', ascending=False)[['final_names','date']].head(10)

KeyError: "['date'] not in index"

In [45]:
results_df

TypeError: 'Index' object is not callable